**Installing Required Dependencies**

In [29]:
!pip install openai==0.28 pandas


**Importing relevant modules**

In [ ]:
import openai
import pandas as pd
import os
import sqlite3

**Setting up Environment Key for OpenAI**

In [ ]:
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.environ["OPENAI_API_KEY"]

**Helper Functions**

In [ ]:
def read_input_file(file_path):
    with open(file_path, 'r') as file: # Open the file located at file_path in read mode ('r')
        questions = file.readlines() # Read all lines from the file into a list
    return [q.strip() for q in questions] # Strip leading/trailing whitespace from each question and return as a list

In [30]:
# Function to fetch table schema dynamically from the database
def fetch_table_schema(connection, table_name):

    ## establishing connection with database
    cursor = connection.cursor()

    ## query to get information of table schema
    query = f"PRAGMA table_info({table_name});"

    ## execute query
    cursor.execute(query)

    ## fetch all results of the query
    schema_info = cursor.fetchall()

    ## close connection to free up resources
    cursor.close()

    ## return column schema
    return [(col[1], col[2]) for col in schema_info]

In [31]:
def generate_sql_query(question, schema, table_name):

    # Convert the schema into a string format for the prompt
    schema_str = ", ".join([f"{col} {dtype}" for col, dtype in schema])

    #prompt for the GPT model, including the schema and the natural language question
    prompt = f"""
            You are an SQL expert. Convert the following natural language question into an SQL query based on the given schema:

            Schema:
            {table_name}({schema_str})

            Question: {question}

            Provide the SQL Query in a concise and correct format, ensuring it matches the schema.
            """
    # Send the prompt to the OpenAI API and get the response
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful coding assistant that converts natural language questions into SQL queries."}, # Provide system-level instructions for the assistant
            {"role": "user", "content": prompt} # Provide the user's input as a message
        ],
        max_tokens=100 # Limit the response length
    )

    # return the SQL query from the API response
    return response.choices[0].message['content'].strip()




In [ ]:
def optimize_sql_query(query):
    # The prompt includes a description and the SQL query that needs to be optimized
    prompt = f"Analyze and optimize the following SQL query for performance improvements:\n\n" \
             f"SQL Query: {query}\n\n" \
             f"Optimized SQL Query and Suggestions:"

    # Make a request to the OpenAI API using the ChatCompletion endpoint
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful coding assistant that optimizes SQL queries."}, # Provide system-level instructions for the assistant
            {"role": "user", "content": prompt} # Provide the user's input as a message
        ],
        max_tokens=250 # Limit the response to a maximum of 150 tokens
    )
    return response.choices[0].message['content'].strip() # contains the assistant's reply with optimization techniques

In [ ]:
# Function to execute SQL query on the SQLite database and print the results
def execute_sql_query(query, connection):
    # Remove code block syntax if present
    query = query.replace("```sql", "").replace("```", "").strip()

    cursor = connection.cursor()
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        for row in results:
            print(row)
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        cursor.close()

**Calling Functions to generate Sql queries based on Natural Language**

In [32]:
# Read natural language questions from a file
questions = read_input_file('/content/natural queries.txt')

# Connect to the SQLite database
connection = sqlite3.connect('/content/employee.db') ### you can establish connection with any of your db similarly, i added the db but you can remotely connect too by setting username, password, host etc


table_name = 'Employee' ### Enter Table Name for which you want queries

# Fetch the schema for the specified table
schema = fetch_table_schema(connection, table_name) ### Fetch the schema

# Iterate over each question, along with its index
for idx, question in enumerate(questions):

  # Print the question number and the question itself
  print(f"Question {idx+1}: {question}")

  # Generate an SQL query from the natural language question
  sql_query = generate_sql_query(question,schema,table_name)


  # Print the generated SQL query
  print(f"Generated SQL Query:\n{sql_query}\n")

  # Execute the generated SQL query and print the results
  print("Results of Generated SQL Query:")
  execute_sql_query(sql_query, connection)
  print("\n")

  # Optimize the generated SQL query for performance improvements
  optimized_query = optimize_sql_query(sql_query)

  # Print the optimized SQL query and any suggestions
  print(f"Optimization Suggestions:\n{optimized_query}\n")

  # Print a separator line for readability
  print("="*80)



Question 1: Show me the top 5 highest-paid employees.
Generated SQL Query:
```sql
SELECT EmployeeID, FirstName, LastName, Salary
FROM Employee
ORDER BY Salary DESC
LIMIT 5;
```

Results of Generated SQL Query:
(21, 'Alex', 'Johnson', 200000)
(2, 'Jane', 'Doe', 95000)
(6, 'David', 'Jones', 95000)
(10, 'Robert', 'Rodriguez', 95000)
(14, 'Richard', 'Gonzalez', 95000)


Optimization Suggestions:
To optimize the given SQL query for better performance, you can consider the following improvements:

1. Add an index on the `Salary` column: Creating an index on the `Salary` column can improve the query's performance when sorting by salary.

2. Limit the columns selected: Since you only need `EmployeeID`, `FirstName`, `LastName`, and `Salary` columns, it's better to only select these specific columns rather than selecting all columns from the `Employee` table.

3. Consider using a covering index: If you frequently need to retrieve these specific columns and sort by salary, you may consider creati